[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain-academy/blob/main/module-0/basics.ipynb) [![Open in LangChain Academy](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66e9eba12c7b7688aa3dbb5e_LCA-badge-green.svg)](https://academy.langchain.com/courses/take/intro-to-langgraph/lessons/56295530-getting-set-up-video-guide)

# LangChain Academy

Welcome to LangChain Academy! 

## 背景

在 LangChain，我们的目标是让大家更容易地构建 LLM 应用程序。其中一种可以构建的 LLM 应用就是智能体（agent）。大家对构建智能体非常兴奋，因为它们可以自动化许多以前无法完成的任务。  

不过在实践中，构建能够可靠执行这些任务的系统非常困难。随着我们帮助用户将智能体投入生产，我们发现往往需要更多的控制。比如，你可能需要智能体在一开始总是调用某个特定工具，或者根据不同的状态使用不同的提示词。  

为了解决这个问题，我们构建了 [LangGraph](https://langchain-ai.github.io/langgraph/) —— 一个用于构建单智能体和多智能体应用的框架。它独立于 LangChain 包，核心设计理念是帮助开发者在智能体工作流中实现更好的精确性与控制，从而适应现实世界系统的复杂性。

## 课程结构

本课程由一系列模块组成，每个模块都专注于 LangGraph 相关的特定主题。你会看到每个模块对应一个文件夹，文件夹中包含一系列 notebook。每个 notebook 都会配有讲解视频，但 notebook 本身也可以独立使用，因为它们包含完整的解释，可以不依赖视频单独阅读。  

每个模块文件夹中还包含一个 `studio` 文件夹，其中包含一组可以加载到 [LangGraph Studio](https://github.com/langchain-ai/langgraph-studio) 的图表，这是我们专门为构建 LangGraph 应用开发的 IDE。

## 环境配置

在开始之前，请按照 `README` 文件中的说明来创建环境并安装依赖。

## 聊天模型

在本课程中，我们会使用 [聊天模型](https://python.langchain.com/v0.2/docs/concepts/#chat-models)。聊天模型的作用是：接收一系列消息作为输入，并返回聊天消息作为输出。LangChain 本身不托管任何聊天模型，而是依赖第三方的集成。[这里](https://python.langchain.com/v0.2/docs/integrations/chat/) 列出了 LangChain 中支持的第三方聊天模型集成！  

默认情况下，本课程会使用 [ChatOpenAI](https://python.langchain.com/v0.2/docs/integrations/chat/openai/)，因为它既流行又高效。请注意，你需要确保已经设置了 `OPENAI_API_KEY`。  

让我们先检查一下是否已经设置了 `OPENAI_API_KEY`，如果没有设置，将会提示你输入。

In [1]:
%%capture --no-stderr
%pip install --quiet -U langchain_core langchain_community tavily-python dashscope

In [2]:
import os, getpass


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


# _set_env("OPENAI_API_KEY")
_set_env("DASHSCOPE_API_KEY")

In [3]:
# from langchain_openai import ChatOpenAI

# gpt4o_chat = ChatOpenAI(model="gpt-4o", temperature=0)
# gpt35_chat = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

from langchain_community.chat_models import ChatTongyi

dashscope_chat = ChatTongyi(model="qwen-plus", temperature=0)

LangChain 中的聊天模型有一些[默认方法](https://python.langchain.com/v0.2/docs/concepts/#runnable-interface)。在大多数情况下，我们会使用以下方法：

* `stream`: 流式返回响应的片段  
* `invoke`: 在输入上调用链  

另外，如前面提到的，聊天模型会接收[消息](https://python.langchain.com/v0.2/docs/concepts/#messages)作为输入。消息包含一个角色（描述是谁在说话）和一个内容属性。后面我们会更深入地讨论这一点，这里先展示基础用法。


In [4]:
from langchain_core.messages import HumanMessage

# Create a message
msg = HumanMessage(content="Hello world", name="Lance")

# Message list
messages = [msg]

# Invoke the model with a list of messages
dashscope_chat.invoke(messages)

AIMessage(content='Hello! How can I assist you today? 😊', additional_kwargs={}, response_metadata={'model_name': 'qwen-plus', 'finish_reason': 'stop', 'request_id': '3b663b89-e87c-4eb1-b17b-3a334483a1e6', 'token_usage': {'input_tokens': 10, 'output_tokens': 11, 'total_tokens': 21, 'prompt_tokens_details': {'cached_tokens': 0}}}, id='run--1b442c35-6129-4a34-aec1-bce835a7278c-0')

我们会得到一个 `AIMessage` 响应。需要注意的是，我们也可以直接用字符串来调用聊天模型。当输入是一个字符串时，它会被转换成一个 `HumanMessage`，然后传递给底层模型。



In [5]:
dashscope_chat.invoke("hello world")

AIMessage(content='Hello! How can I assist you today? 😊', additional_kwargs={}, response_metadata={'model_name': 'qwen-plus', 'finish_reason': 'stop', 'request_id': 'ddd38dba-24da-4815-9e2a-b2d6d33ad968', 'token_usage': {'input_tokens': 10, 'output_tokens': 11, 'total_tokens': 21, 'prompt_tokens_details': {'cached_tokens': 0}}}, id='run--706d7233-612a-4a17-8525-8e5c918e722d-0')

In [9]:
dashscope_chat.stream("hello world")

<generator object BaseChatModel.stream at 0x10d03c740>

该接口在所有聊天模型中都是一致的，并且模型通常会在每个 notebook 启动时初始化一次。  

因此，如果你更偏好其他提供商，可以轻松切换模型，而无需更改下游代码。



## 搜索工具

你还会在 README 中看到 [Tavily](https://tavily.com/)，它是一个为 LLM 和 RAG 优化的搜索引擎，旨在提供高效、快速且持久的搜索结果。正如前面提到的，它注册非常简单，并且提供了慷慨的免费套餐。  

在某些课程（模块 4）中会默认使用 Tavily，不过当然，如果你想修改代码，也可以使用其他搜索工具。


In [6]:
_set_env("TAVILY_API_KEY")

In [7]:
from langchain_community.tools.tavily_search import TavilySearchResults

tavily_search = TavilySearchResults(max_results=3)
search_docs = tavily_search.invoke("What is LangGraph?")

/var/folders/zr/zsgg61wn27zbcvccfss34hpw0000gq/T/ipykernel_41556/1344879509.py:3: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tavily_search = TavilySearchResults(max_results=3)


In [8]:
search_docs

[{'title': 'What is LangGraph? - Analytics Vidhya',
  'url': 'https://www.analyticsvidhya.com/blog/2024/07/langgraph-revolutionizing-ai-agent/',
  'content': 'To sum up, LangGraph is a major advancement in the development of AI agents. It enables developers to push the limits of what’s possible with AI agents by eliminating the shortcomings of earlier systems and offering a flexible, graph-based framework for agent construction and execution. LangGraph is positioned to influence the direction of artificial intelligence significantly in the future. [...] LangGraph is a library built on top of Langchain that is designed to facilitate the creation of cyclic graphs for large language model (LLM) – based AI agents.\n It views agent Objective Points about LangGraph and workflows as cyclic graph topologies, allowing for more variable and nuanced agent behaviors than linear execution models. [...] Frameworks such as LangGraph are becoming increasingly important as AI develops. LangGraph is mak